In [1]:
from __future__ import print_function
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable

import os
import csv
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload

%autoreload 2
plt.ion()   # interactive mode

In [2]:
# Prepare the Dataset
DATA_DIR = "Dataset/trainingSet/"
with open('Dataset/Processed/MNIST_Dataset.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["filename", "label"])
    for label in xrange(0, 9):
        path = os.path.join(DATA_DIR, str(label))
        for p in os.listdir(path):
            filename = os.path.join(path, p)
            writer.writerow([filename, str(label)])
            
data = pd.read_csv("Dataset/Processed/MNIST_Dataset.csv")
data = data.iloc[np.random.permutation(len(data))]
train, validation = data[:25000],data[25000:]
train.to_csv("Dataset/Processed/MNIST_Train.csv", index=False)
validation.to_csv("Dataset/Processed/MNIST_Validation.csv", index=False)

In [3]:
from dataset import MNISTDataset
mnist_train_dataset = MNISTDataset(csv_file='./Dataset/Processed/MNIST_Train.csv', root_dir='./')
mnist_val_dataset = MNISTDataset(csv_file='./Dataset/Processed/MNIST_Validation.csv', root_dir='./')
train_dataloader = DataLoader(mnist_train_dataset, batch_size=1000, shuffle=True, num_workers=4)
val_dataloader = DataLoader(mnist_val_dataset, batch_size=1000, shuffle=True, num_workers=4)

In [4]:
from model import Net 
model_instance = Net()
v = Variable(torch.randn(100,1,28,28))
model_instance.forward(v)

torch.Size([100, 64, 7, 7])


Variable containing:
-2.1969 -2.3444 -2.2885  ...  -2.2327 -2.4185 -2.4122
-2.1640 -2.3358 -2.3477  ...  -2.3682 -2.4241 -2.3318
-2.1569 -2.2267 -2.3261  ...  -2.3485 -2.3775 -2.2381
          ...             ⋱             ...          
-2.0662 -2.2726 -2.3799  ...  -2.4549 -2.2546 -2.3350
-2.1078 -2.3632 -2.2318  ...  -2.3085 -2.3213 -2.2334
-2.1979 -2.3383 -2.3038  ...  -2.3128 -2.3045 -2.3772
[torch.FloatTensor of size 100x10]

In [6]:
for p in model_instance.parameters():
    print(p.size())

torch.Size([32, 1, 5, 5])
torch.Size([32])
torch.Size([64, 32, 5, 5])
torch.Size([64])
torch.Size([1024, 3136])
torch.Size([1024])
torch.Size([10, 1024])
torch.Size([10])


In [11]:
optimizer = optim.Adam(model_instance.parameters(), lr=0.0001)

In [8]:
def train(model, epoch):
    model.train()
    correct = 0
    for batch_idx, data in enumerate(train_dataloader):
        data, target = Variable(data["image"]), Variable(data["label"])
        optimizer.zero_grad()
        output = model.forward(data.float())
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        if batch_idx % 1 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_dataloader.dataset),
                100. * batch_idx / len(train_dataloader), loss.data[0]))
    print('Train accuracy : {:.6f}'.format(100. * correct / len(train_dataloader.dataset)))

In [9]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data in val_dataloader:
        data, target = Variable(data["image"], volatile=True), Variable(data["label"])
        output = model(data.float())
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(val_dataloader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(val_dataloader.dataset),
        100. * correct / len(val_dataloader.dataset)))


In [12]:
# 99 % validation accuracy 
for epoch in range(1, 15):
    train(model_instance, epoch)
    test(model_instance)    

torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [0/24999 (0%)]	Loss: 1.198431
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [1000/24999 (4%)]	Loss: 1.709447
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [2000/24999 (8%)]	Loss: 0.862743
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [3000/24999 (12%)]	Loss: 1.119872
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [4000/24999 (16%)]	Loss: 0.841571
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [5000/24999 (20%)]	Loss: 0.624258
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [6000/24999 (24%)]	Loss: 0.445514
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [7000/24999 (28%)]	Loss: 0.520960
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [8000/24999 (32%)]	Loss: 0.518331
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [9000/24999 (36%)]	Loss: 0.531566
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [10000/24999 (40%)]	Loss: 0.376771
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [11000/24999 (44%)]	Loss: 0.357228
torch.Size([1000, 64, 7, 7])
Train Epoch: 1 [12000/24999 (48%)]	Loss

KeyboardInterrupt: 

In [14]:
torch.save(model_instance, './saved_models/net_c32xc64xf3136xf1024.pth')